# Combine Labels

In [ ]:
# import and constant
import os
import numpy as np
import nibabel as nib
from utils.nrrd2nii import nrrd2nii

source_path = "data/uwa_aaa"
combined_path = "data/nnunet_raw/Dataset001_raw_aaa"
os.makedirs(f"{combined_path}/imagesTr", exist_ok=True)
os.makedirs(f"{combined_path}/labelsTr", exist_ok=True)
os.makedirs(f"{combined_path}/imagesTs", exist_ok=True)
os.makedirs(f"{combined_path}/labelsTs", exist_ok=True)

# History Reseason to keep order
origin_list = os.listdir("data/uwa_aaa")
for i, s in enumerate(origin_list):
    origin_list[i] = s + "_"
origin_list = sorted(origin_list)
count = 0
for i, s in enumerate(origin_list):
    origin_list[i] = s.replace("_", "")


for patient_folder in origin_list:
    patient_path = os.path.join(source_path, patient_folder)
    aaa_label = None
    blood_label = None
    cropped_image = None
    for root, dirs, files in os.walk(patient_path):
        for file in files:
            if file == "CT_AAA_label.nrrd":
                aaa_label = nrrd2nii(os.path.join(root, file)).get_fdata()

                assert (
                    len(np.unique(aaa_label)) == 2
                ), f"Patient {patient_folder} label error"

                highest_label = np.max(aaa_label)
                aaa_label[aaa_label == highest_label] = 1

            if file == "CT_blood_label.nrrd":
                blood_label = nrrd2nii(os.path.join(root, file)).get_fdata()
                
                assert (
                    len(np.unique(blood_label)) == 2
                ), f"Patient {patient_folder} label error"

                highest_label = np.max(blood_label)
                blood_label[blood_label == highest_label] = 1
            
            if file == "CT_cropped.nrrd":
                cropped_image = nrrd2nii(os.path.join(root, file)).get_fdata()


    if (aaa_label is None) or (blood_label is None) or (cropped_image is None):
        print(f"Patient {patient_folder} data incomplete")
        continue

    if aaa_label.shape != blood_label.shape or aaa_label.shape != cropped_image.shape:
        print(f"Patient {patient_folder} label shape error: {aaa_label.shape} vs {blood_label.shape} vs {cropped_image.shape}")
        continue
    
    if (blood_label.shape[0] * blood_label.shape[1] * blood_label.shape[2]) > 10000000:
        print(f"Patient {patient_folder} too large: {blood_label.shape}")
        continue

    combined_label = np.zeros_like(aaa_label)
    combined_label[aaa_label == 1] = 1
    combined_label[blood_label == 1] = 2
    
    # if shape contains singular, make it even
    padding = []
    for dim in combined_label.shape:
        if dim % 2 == 1:
            padding.append(1)
        else:
            padding.append(0)
    combined_label = np.pad(
        combined_label,
        ((0, padding[0]), (0, padding[1]), (0, padding[2])),
        mode="constant",
    )
    cropped_image = np.pad(
        cropped_image,
        ((0, padding[0]), (0, padding[1]), (0, padding[2])),
        mode="constant",
    )
    if count < 16:
        label_nii = nib.Nifti1Image(combined_label, np.eye(4))
        nib.save(label_nii, os.path.join(f"{combined_path}/labelsTr", f"aaa_{count:03d}.nii.gz"))
        image_nii = nib.Nifti1Image(cropped_image, np.eye(4))
        nib.save(image_nii, os.path.join(f"{combined_path}/imagesTr", f"aaa_{count:03d}_0000.nii.gz"))
    else:
        label_nii = nib.Nifti1Image(combined_label, np.eye(4))
        nib.save(label_nii, os.path.join(f"{combined_path}/labelsTs", f"aaa_{count:03d}.nii.gz"))
        image_nii = nib.Nifti1Image(cropped_image, np.eye(4))
        nib.save(image_nii, os.path.join(f"{combined_path}/imagesTs", f"aaa_{count:03d}_0000.nii.gz"))
    count += 1

Patient Patient19 too large: (231, 292, 430)
Patient Patient21 data incomplete
Patient Patient22 data incomplete
Patient Patient23 data incomplete
Patient Patient24 data incomplete
Patient Patient25 data incomplete
Patient Patient27 data incomplete
Patient Patient28 data incomplete
Patient Patient29 data incomplete
Patient Patient2 label shape error: (138, 123, 151) vs (138, 123, 200) vs (138, 123, 151)
Patient Patient30 data incomplete
Patient Patient31 data incomplete
Patient Patient4 label shape error: (152, 157, 171) vs (152, 157, 95) vs (152, 157, 72)
Patient Patient7 label shape error: (126, 135, 147) vs (512, 512, 239) vs (126, 135, 147)


# Inspeact labels

In [25]:
from utils.check_image import check_image
import nibabel as nib
from utils.nrrd2nii import nrrd2nii

image = nib.load("data/nnunet_raw/Dataset001_raw_aaa/imagesTr/aaa_000_0000.nii.gz").get_fdata()
label = nib.load("data/nnunet_raw/Dataset001_raw_aaa/labelsTr/aaa_000.nii.gz").get_fdata()

In [26]:
check_image(
    [
        {"image": image, "title": "image", "is_label": False},
        {"image": label, "title": "combined", "is_label": True},
    ]
)

interactive(children=(IntSlider(value=0, description='slice_index_0', max=98), IntSlider(value=0, description=…